In [1]:
from HELPpy.visualization.filecollector import FileCollector
import ipywidgets as wid
fc = FileCollector(".", title='Choose file', filter_pattern='*.csv', layout=wid.Layout(width='auto'))
display(fc)

FileCollector(path='/Users/maurizio/HELP/prova', filename='()', title='Choose file', show_hidden=False, select…

In [20]:
import os
s = ('/Users/maurizio/HELP/HELPpy/notebooks/Kidney_AE.csv', '/Users/maurizio/HELP/HELPpy/notebooks/myfilechooser.py', '/Users/maurizio/HELP/HELPpy/notebooks/Kidney_BPBeder_0.csv', '/Users/maurizio/HELP/HELPpy/notebooks/Kidney_BPBeder_10.csv', '/Users/maurizio/HELP/HELPpy/notebooks/Kidney_BIO_w_const.csv', '/Users/maurizio/HELP/HELPpy/notebooks/Kidney_BPBeder_1.csv', '/Users/maurizio/HELP/HELPpy/notebooks/prova.ipynb')
[os.path.isfile(ss) for ss in s]

[True, False, False, False, False, False, False]

In [44]:
import ipywidgets as widgets
widget = widgets.HTML(value= "a \n b" +' </p>')
display(widget)

HTML(value='a \n b </p>')

In [21]:
from tqdm import tqdm

class ipytqdm(tqdm):
    """store file metadata"""
    def __init__(self, pbar=None, **kwds):
        self.ipybar = pbar
        if self.ipybar is not None:
            self.ipybar.max = self.total
        super().__init__(**kwds)
    def update(self, n=1):
        if self.ipybar is not None:
            self.ipybar.value += n
        else:
            super().update(n)

 
from ipywidgets import IntProgress
display(pb)

pb = IntProgress(min=0, value=0)
with ipytqdm(total=int(9e6), pbar=pb) as bar:
    for i in range(int(9e6)):
        bar.update()



IntProgress(value=100)

NameError: name 'sel' is not defined